From this program the user can input the following derived KPIs and  behavioural parameters of lines and generate events events that results those derived KPIs.



**Inputs**
*   Operators
*   Helpers
*   Ironman
*   QC
*   Schedule showing work hour length
*   SMV
*   Efficiency
*   Defective rate


**Behavioural hyperparametrs you can adjust**
*   ftt_ratio : Ratio of FTTs from FTT+ Rectified ina  line
*   direct_rejects : Number of pcs being directly rejeceted without attempted rectification
*   defect_increment : Difference between the number of defective and defects
*   target_increment : Offset to add to FTT + Rectified + Reject to generate
planned_qty


**KPIs being fixed as a result**

*   Manpower
*   Man Minutes
*   SAH
*   Actual output
*   Trend
*   Checked_pcs
*   FTT
*   Reworked
*   Defective
*   Reject
*   Checked pcs
*   FTT %
*   Reject rate%
*   DHU
*   Planned Qty
*   RTT





In [12]:
# all imports
import pandas as pd
import numpy as np
import random
from random import randint
from google.colab import drive

**Note that the plan_qty and produced_mins columns in the plan are going to be altered according to the derived KPIs you provide in a later stage.**

In [13]:
# Read plan and schedule data from drive
drive.mount('/content/drive')
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
plan

,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,2106,660,23,5,0,0,28,8.28,17437.68
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1849,660,23,5,0,0,28,9.45,17473.05
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,1797,660,20,5,0,0,25,8.28,14879.16
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,1796,660,23,5,0,0,28,10.11,18157.56
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,2179,660,23,5,0,0,28,8.21,17889.59
5,12f0ffe7-c509-49b5-b203-83be908b2886,2,2024-07-31 10:00:00,2024-07-31 14:00:00,2024-07-31,1,Morning,NaN,NaN,NaN,...,Black,1852,660,23,5,0,0,28,9.45,17501.40
6,12f0ffe7-c509-49b5-b203-83be908b2887,2,2024-08-01 10:00:00,2024-08-01 13:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,...,Print,2692,660,26,8,0,0,34,8.28,22289.76
7,13f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,1202,660,12,5,0,0,17,8.28,9952.56
8,13f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1126,660,12,5,0,0,17,9.45,10640.70
9,13f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,1120,660,10,5,0,0,15,8.28,9273.60


In [15]:
# Generate the skeleton of the target table
repeated_plan_date =  plan['date'].repeat(len(schedule)).reset_index(drop=True)
repeated_module_id = plan['module_id'].repeat(len(schedule)).reset_index(drop=True)
hours = list(range(len(schedule)+1))
hours.remove(0)
repeated_hour = hours*len(plan)
targets = pd.DataFrame({'Date': repeated_plan_date, 'Line' : repeated_module_id,
                        'Hour': repeated_hour})


Let us generate reasonable manpower-realated columns by refering the plan data.
**Remember to change the Manpower definition according to the factory.**

In [16]:
def generate_man_power(targets, plan, manpower_regulator):
    """
    Generate counts for each hour of each day for each line group of a DataFrame
    to match the target counts by distributing the target sum among all rows in
    the hour + date + line group.

    :param target: DataFrame containing targets.
    :param plan: DataFrame with planned values per day.
    :param manpower_regulator : The higest possible member variation
    :return: DataFrame with generated manpower counts.
    """
    # Merge the dataframes on the 'hour, date, line' columns
    merged_df = targets.merge(plan[['date', 'module_id',  'operator', 'helper',
       'ironman', 'qc', 'manpower', 'smv']],
                              left_on=['Date', 'Line'],
                              right_on= ['date','module_id'], how='left')
    merged_df.drop(columns=['date', 'module_id'], inplace=True)


    merged_df['Operator'] = np.maximum((merged_df['operator'] - np.random.choice([0,manpower_regulator],
                                          len(merged_df), p=[0.95, 0.05])).astype(int),0)
    merged_df['Helper'] = np.maximum((merged_df['helper'] - np.random.choice([0,manpower_regulator],
                                          len(merged_df), p=[0.95, 0.05])).astype(int),0)
    merged_df['Iron man'] = np.maximum((merged_df['ironman'] - np.random.choice([0,manpower_regulator],
                                          len(merged_df), p=[0.95, 0.05])).astype(int),0)
    merged_df['QC'] = np.maximum((merged_df['qc'] - np.random.choice([0,manpower_regulator],
                                          len(merged_df), p=[0.95, 0.05])).astype(int),0)
    # Define this as factory defines
    merged_df['Man power'] = merged_df['Operator'] + merged_df['Helper']
    merged_df.drop(columns=['operator', 'helper', 'ironman', 'qc','manpower'], inplace=True)
    return merged_df



In [17]:
merged_df = generate_man_power(targets,plan, manpower_regulator=1)
merged_df.head(100)

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power
0,2024-07-26,F1M7,1,8.28,23,5,0,0,28
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28
4,2024-07-26,F1M7,5,8.28,22,5,0,0,27
...,...,...,...,...,...,...,...,...,...
95,2024-07-27,F1M8,8,9.45,12,5,0,0,17
96,2024-07-27,F1M8,9,9.45,12,5,0,0,17
97,2024-07-27,F1M8,10,9.45,12,5,0,0,17
98,2024-07-27,F1M8,11,9.45,12,5,0,0,17


In [18]:
#Convert start_time and end_time to datetime
schedule['start_time'] = pd.to_datetime(schedule['start_time'], format='%H:%M:%S')
schedule['end_time'] = pd.to_datetime(schedule['end_time'], format='%H:%M:%S')

# Calculate the duration between start_time and end_time in hours
schedule['clock_mins'] = (schedule['end_time'] - schedule['start_time']).dt.total_seconds() / 60
schedule

# Sum the clock_hours to get total hours in clock time
# total_clock_hours = schedule['clock_hours'].sum()
# total_clock_hours

,hour,start_time,end_time,clock_mins
0,1,1900-01-01 07:30:00,1900-01-01 08:30:00,60.0
1,2,1900-01-01 08:30:00,1900-01-01 09:30:00,60.0
2,3,1900-01-01 09:30:00,1900-01-01 10:30:00,60.0
3,4,1900-01-01 10:30:00,1900-01-01 11:30:00,60.0
4,5,1900-01-01 11:30:00,1900-01-01 13:00:00,90.0
5,6,1900-01-01 13:00:00,1900-01-01 14:00:00,60.0
6,7,1900-01-01 14:00:00,1900-01-01 15:00:00,60.0
7,8,1900-01-01 15:00:00,1900-01-01 16:00:00,60.0
8,9,1900-01-01 16:00:00,1900-01-01 17:00:00,60.0
9,10,1900-01-01 17:00:00,1900-01-01 18:00:00,60.0


In [19]:
# Now we merge the schedule with the merged_df to find the length of each working hour
derived_kpis = merged_df.merge(schedule[['hour', 'clock_mins']],
                              left_on=['Hour'],
                              right_on= ['hour'], how='left')
derived_kpis.drop(columns=['hour'], inplace=True)
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins
0,2024-07-26,F1M7,1,8.28,23,5,0,0,28,60.0
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60.0
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60.0
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60.0
4,2024-07-26,F1M7,5,8.28,22,5,0,0,27,90.0
...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60.0
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60.0
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60.0
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60.0


In [20]:
np.random.seed(42)

Now let us add an efficiency column with values between 70-100 and a defective_rate column between 3-12.

**If needed the user can go and change this in the sheet.**
ftt_ratio indicates the propotion of ftts from ftt+ rectified qty.
( that is, ftt/ all actuals)



In [21]:
# Autogenerate efficiency, defective rate and ftt_ratio

derived_kpis['target_efficiency'] = np.random.randint(70, 100, size=len(derived_kpis))
derived_kpis['target_defective_rate'] = np.random.randint(3, 20, size=len(derived_kpis))
derived_kpis['ftt_ratio'] = np.random.uniform(0.8, 0.98, size=len(derived_kpis))
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,target_efficiency,target_defective_rate,ftt_ratio
0,2024-07-26,F1M7,1,8.28,23,5,0,0,28,60.0,76,11,0.887614
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60.0,89,10,0.963098
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60.0,98,14,0.878191
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60.0,84,4,0.863014
4,2024-07-26,F1M7,5,8.28,22,5,0,0,27,90.0,80,3,0.916119
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60.0,76,14,0.922690
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60.0,95,16,0.861312
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60.0,78,8,0.846925
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60.0,97,8,0.889287


In [22]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='derived_kpis', index=False)